# ***Matrix Profile*** (STUMP)

In this notebook, the Matrix Profile is evaluated. The implementation is the STUMP function of the stumpy module.

## Imports

In [6]:
import math
import os
import sys
import json
from time import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

In [7]:
from TSB_UAD.models.distance import Fourier
from TSB_UAD.models.feature import Window
from TSB_UAD.utils.slidingWindows import find_length, plotFig, printResult

from TSB_UAD.models.iforest import IForest

## Data Pre-Processing

In [8]:
# Load the data for the evaluation.
all_data = []

with open('dataset.pkl', 'rb') as f:
    data = pkl.load(f)

all_data.extend(data['evaluation']['single_normality'])
all_data.extend(data['evaluation']['double_normality'])
all_data.extend(data['evaluation']['triple_normality'])
all_data.extend(data['evaluation']['quadruple_normality'])

In [9]:
preprocessed_dict = {}

In [52]:
# Set the number of windows to be fit per batch.
from typing import Any


windows_per_batch = 150

for timeseries in all_data:
    
    # === Pre-processing steps ===

    # Prepare data for unsupervised method
    name = timeseries['Name']

    data = timeseries['data']
    max_length = data.shape[0]
    label = timeseries['labels']

    slidingWindow = find_length(data)
    X_data = Window(window=slidingWindow).convert(data).to_numpy()

    # Take the series and batch it.
    batched_data = []

    i = 0
    flag = True
    # Keep taking batches until the point at which no new windows can be taken.
    while i < len(data) and flag:
        # The data batches begin at the index indicated. If first batch, then the beginning of the time series.
        batch_samples_begin = i

        # The data batches end at the index where `windows_per_batch` can be *completely* extracted since the batch beginning. 
        # Formula: 
        #   i: current beginning of batch / offset
        #   + slidingWindow: to have enough samples extract one window
        #   + windows_per_batch: to have enough samples to extract the rest of the windows
        #   - 1: because the first window extracted is counted twice
        batch_samples_end = i + windows_per_batch + slidingWindow - 1
        
        # Guard against the ending of the time series where a full batch cannot be formed.
        if batch_samples_end > len(data):
            batch_samples_end = len(data)
            flag = False
 
        # Guard against case where the batch cannot hold even one window.
        if len(data[batch_samples_begin:batch_samples_end]) < slidingWindow:
            break

        batched_data.append(data[batch_samples_begin:batch_samples_end])

        # The next batch starts at the point where a new window be created after the last window of the last batch.
        # So, end of the previous window - length of window = start of the last window.
        #   start of the last window + 1 = start of the first window of the next batch.
        i = batch_samples_end - slidingWindow + 1


    # Take the series and batch it for history batching: For each batch, also append have access to the data of the previous batch.
    batched_data_previous_access = []
    i = 0
    previous_window_beginning = 0
    flag = True
    # Keep taking batches until the point at which no new windows can be taken.
    while i < len(data) and flag:
        # The data batches begin at the index indicated. If first batch, then the beginning of the time series.
        batch_samples_begin = i

        # The data batches end at the index where `windows_per_batch` can be *completely* extracted since the batch beginning. 
        # Formula: 
        #   i: current beginning of batch / offset
        #   + slidingWindow: to have enough samples extract one window
        #   + windows_per_batch: to have enough samples to extract the rest of the windows
        #   - 1: because the first window extracted is counted twice
        batch_samples_end = i + windows_per_batch + slidingWindow - 1
        
        # Guard against the ending of the time series where a full batch cannot be formed.
        if batch_samples_end > len(data):
            batch_samples_end = len(data)
            flag = False
 
        # Guard against case where the batch cannot hold even one window.
        if len(data[batch_samples_begin:batch_samples_end]) < slidingWindow:
            break

        batched_data_previous_access.append(data[previous_window_beginning:batch_samples_end])

        previous_window_beginning = batch_samples_begin

        # The next batch starts at the point where a new window be created after the last window of the last batch.
        # So, end of the previous window - length of window = start of the last window.
        #   start of the last window + 1 = start of the first window of the next batch.
        i = batch_samples_end - slidingWindow + 1

    # Take the windows and batch them.
    batched_X_data = []
    i = 0
    while i < len(X_data):
        begin = i
        end = i + windows_per_batch
        if end > len(X_data):
            end = len(X_data)

        batched_X_data.append(X_data[begin:end])
        i += windows_per_batch


    # Processing data for dynamic partitioning.
    initial_partition_length = int(len(data) * 0.25)
    initial_partition = data[:initial_partition_length]

    max_v = np.max(initial_partition)
    min_v = np.min(initial_partition)

    data_partitions = [initial_partition]
    current_partition = []
    change_detected = False

    p = 500
    change_point_threshold = 0.5
    exceed_threshold = 0.65
    post_change_points = []

    for point in data[initial_partition_length:]:
        
        # Check for significant change
        if (point > max_v * (1 + change_point_threshold)) or (point < min_v * (1 - change_point_threshold)):
            change_detected = True
     
        current_partition.append(point)


        # After change, collect additional points
        if change_detected:
            post_change_points.append(point)
            if len(post_change_points) == p:
                exceeds_threshold_points = [(pt > max_v * (1 + change_point_threshold) or pt < min_v * (1 - change_point_threshold)) for pt in post_change_points]
                if sum(exceeds_threshold_points) >= exceed_threshold * p:
                    max_v = np.mean([max_v] + [pt for pt in post_change_points if pt > max_v])
                    min_v = np.mean([min_v] + [pt for pt in post_change_points if pt < min_v])

                post_change_points = []

                # Add the current partition to data partitions
                data_partitions.append(np.array(current_partition))
                current_partition = []
                change_detected = False
                
        
    # Add any remaining points in current_partition to data_partitions
    if current_partition:
        data_partitions.append(np.array(current_partition))

    # Processing data for dynamic partitioning (Percentile variant)
    initial_partition_length = int(len(data) * 0.25)
    initial_partition = data[:initial_partition_length]

    # Compute initial percentiles
    percentile_5 = np.percentile(initial_partition, 5)
    percentile_95 = np.percentile(initial_partition, 95)

    percentile_data_partitions = []
    current_partition = []
    change_detected = False
    p = 500
    exceed_threshold = 0.5
    post_change_points = []

    for point in data[initial_partition_length:]:
        
        # Check for significant change
        if (point < percentile_5) or (point > percentile_95):
            change_detected = True
     
        current_partition.append(point)


        # After change, collect additional points
        if change_detected:
            post_change_points.append(point)
            if len(post_change_points) == p:
                exceeds_threshold_points = [(pt < percentile_5 or pt > percentile_95) for pt in post_change_points]
                if sum(exceeds_threshold_points) / p >= exceed_threshold:
                    # Update percentiles
                    percentile_5 = np.percentile(post_change_points, 5)
                    percentile_95 = np.percentile(post_change_points, 95)

                post_change_points = []
                # Add the current partition to data partitions
                percentile_data_partitions.append(np.array(current_partition))
                current_partition = []
                change_detected = False
                
        
    # Add any remaining points in current_partition to percentile_data_partitions
    if current_partition:
        percentile_data_partitions.append(np.array(current_partition))


    print(f'Time-Series name: {name}')
    print("Estimated Subsequence length: ", slidingWindow)
    print()
    
    # Store the pre-processed variables in the new dictionary
    preprocessed_dict[name] = {
        'name': name,
        'data': data,
        'label': label,
        'slidingWindow': slidingWindow,
        'X_data': X_data,
        'batched_X_data': batched_X_data,
        'batched_data': batched_data,
        'points_per_batch': len(batched_data[0]),
        'history_batched_data': batched_data_previous_access,
        'dynamic_partitioning_batches': data_partitions,
        'percentile_dynamic_partitioning_batches': percentile_data_partitions,
        'Time series length': len(data),
        'Number of abnormal points': list(label).count(1)
    }

Time-Series name: ECG1
Estimated Subsequence length:  100

Time-Series name: ECG1_20k
Estimated Subsequence length:  100

Time-Series name: IOPS1
Estimated Subsequence length:  288

Time-Series name: SMD1
Estimated Subsequence length:  125

Time-Series name: Occupancy1
Estimated Subsequence length:  125

Time-Series name: ECG1+IOPS1
Estimated Subsequence length:  100

Time-Series name: SMD1+Occupancy1
Estimated Subsequence length:  125

Time-Series name: ECG1+IOPS1+Occupancy1
Estimated Subsequence length:  100

Time-Series name: SMD1+ECG1+Occupancy1
Estimated Subsequence length:  125

Time-Series name: ECG1+IOPS1+SMD1+Occupancy1
Estimated Subsequence length:  100



## 1. Offline Running
The STUMP algorithm is run offline, with access to all subsequences.

In [12]:
import stumpy
modelName = 'STUMP'

In [16]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    x = ts['data']
    
    k = 1
    t0 = time()
    score_ = stumpy.stump(T_A=x, m=window_size, k=k, ignore_trivial=True, normalize=True)
    t1 = time()
    score = score_.T[k-1]
    
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [ ]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.635414,0.084577,21105.0,61.204756,229900
1,ECG1_20k,0.857905,0.133333,675.0,0.289800,20000
2,IOPS1,0.719323,0.189320,206.0,0.278842,8784
3,SMD1,0.472878,0.000000,2694.0,0.924781,28479
4,Occupancy1,0.169576,0.000000,972.0,0.073836,2665
5,ECG1+IOPS1,0.759207,0.098751,881.0,0.963221,28784
6,SMD1+Occupancy1,0.377355,0.000000,3666.0,1.070978,31144
7,ECG1+IOPS1+Occupancy1,0.765244,0.058284,1853.0,1.074390,31449
8,SMD1+ECG1+Occupancy1,0.576887,0.000000,4341.0,2.025657,51144
9,ECG1+IOPS1+SMD1+Occupancy1,0.613329,0.000000,4547.0,2.503679,59928


In [ ]:
df.to_csv('Results/Static/STUMP-results.csv', index=False)

## 2. Online Running: Variant 1
The STUMP algorithm is run online, with access to limited subsequences. In this variant, the algorithm only has access to the subsequences of a given (current) batch.

In [ ]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    x = ts['data']
    
    k = 1
    score = []
    t0 = time()
    for batch in tqdm(ts['batched_data'], desc='Processing Batch'):
        score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)
        score.extend(score_.T[k-1])
    t1 = time()

    # In some combinations of batch size and window size, windows overlap with all closest-distance candidates and cannot be scored.
    # In this case, inf is returned. To fix this, any instances of infinite distances are replaced with zero distance.
    score = [s if s != np.inf else 0 for s in score]
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1+IOPS1+SMD1+Occupancy1: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it]


In [ ]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [ ]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.696418,0.0,21105.0,11.055027,59928
1,ECG1_20k,0.689407,0.0,675.0,0.964218,59928
2,IOPS1,0.596923,0.0,206.0,0.417511,59928
3,SMD1,0.556622,0.0,2694.0,1.334138,59928
4,Occupancy1,0.186049,0.0,972.0,0.126538,59928
5,ECG1+IOPS1,0.708663,0.0,881.0,1.368074,59928
6,SMD1+Occupancy1,0.441063,0.0,3666.0,0.943233,59928
7,ECG1+IOPS1+Occupancy1,0.499768,0.0,1853.0,0.934797,59928
8,SMD1+ECG1+Occupancy1,0.542541,0.0,4341.0,1.619330,59928
9,ECG1+IOPS1+SMD1+Occupancy1,0.525118,0.0,4547.0,1.851899,59928


In [ ]:
df.to_csv('Results/Variant_1/STUMP-results.csv', index=False)

## 3. Online Running: Variant 2: History Batching
The STUMP algorithm is run online, with access to limited subsequences. In this variant, the algorithm only has access to the subsequences of a given (current) batch and the previous batch.

In [58]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    x = ts['data']
    
    k = 1
    score = []
    t0 = time()

    for i, batch in enumerate(tqdm(ts['history_batched_data'], desc='Processing Batch')):
        score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)

        if i == 0:
            next_scores = score_.T[k-1]
        else:
            new_scores = list(score_.T[k-1])[:windows_per_batch]
            next_scores = list(score_.T[k-1])[windows_per_batch:]
        
            old_scores_new_scores_mean = list(np.add(previous_scores, new_scores)/2)
            score.extend(old_scores_new_scores_mean)

        previous_scores = next_scores
    # The final batch's scores don't have any future to average with, they are just added.
    score.extend(previous_scores)
    t1 = time()

    # In some combinations of batch size and window size, windows overlap with all closest-distance candidates and cannot be scored.
    # In this case, inf is returned. To fix this, any instances of infinite distances are replaced with zero distance.
    score = [s if s != np.inf else 0 for s in score]
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1+IOPS1+SMD1+Occupancy1: 100%|██████████| 10/10 [00:24<00:00,  2.46s/it]


In [59]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [60]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.845447,0.114949,21105.0,9.504750,229900
1,ECG1_20k,0.968435,0.299259,675.0,0.842503,20000
2,IOPS1,0.460942,0.000000,206.0,0.356547,8784
3,SMD1,0.464620,0.000000,2694.0,1.133917,28479
4,Occupancy1,0.190969,0.000000,972.0,0.100159,2665
5,ECG1+IOPS1,0.901091,0.138479,881.0,1.232482,28784
6,SMD1+Occupancy1,0.374143,0.000000,3666.0,1.240844,31144
7,ECG1+IOPS1+Occupancy1,0.684548,0.072315,1853.0,1.296250,31449
8,SMD1+ECG1+Occupancy1,0.569588,0.000000,4341.0,2.154027,51144
9,ECG1+IOPS1+SMD1+Occupancy1,0.590303,0.000000,4547.0,2.386611,59928


In [61]:
print(df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']].to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                      Name &      AUC &  Precision@k &  Number of anomalies &     Time &  Number of Windows \\
\midrule
                      ECG1 & 0.845447 &     0.114949 &              21105.0 & 9.504750 &             229900 \\
                  ECG1\_20k & 0.968435 &     0.299259 &                675.0 & 0.842503 &              20000 \\
                     IOPS1 & 0.460942 &     0.000000 &                206.0 & 0.356547 &               8784 \\
                      SMD1 & 0.464620 &     0.000000 &               2694.0 & 1.133917 &              28479 \\
                Occupancy1 & 0.190969 &     0.000000 &                972.0 & 0.100159 &               2665 \\
                ECG1+IOPS1 & 0.901091 &     0.138479 &                881.0 & 1.232482 &              28784 \\
           SMD1+Occupancy1 & 0.374143 &     0.000000 &               3666.0 & 1.240844 &              31144 \\
     ECG1+IOPS1+Occupancy1 & 0.684548 &     0.072315 &               

In [62]:
df.to_csv('Results/Matrix Profile/STUMP-variant2', index=False)

## 4. Online Running: Variant 3: Dynamic Partitioning.
The STUMP algorithm is run online, with access to limited subsequences. In this variant, the algorithm only has access to the subsequences of a given (current) batch. The partitioning of the original time series is being done dynamically.

In [53]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    x = ts['data']
    
    k = 1
    score = []
    t0 = time()
    for i, batch in enumerate(tqdm(ts['dynamic_partitioning_batches'], desc='Processing Batch')):
        score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)
        score_ = score_.T[k-1]

        # Because batches are being split in a way that doesn't allow windows to cross batch boundaries, pad predictions to account for lost
        # windows.
        score_ = np.array([score_[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score_) + [score_[-1]]*((ts['slidingWindow']-1)//2))

        score.extend(score_)
    t1 = time()

    # In some combinations of batch size and window size, windows overlap with all closest-distance candidates and cannot be scored.
    # In this case, inf is returned. To fix this, any instances of infinite distances are replaced with zero distance.
    score = [s if s != np.inf else 0 for s in score]
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    # score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1+IOPS1+SMD1+Occupancy1: 100%|██████████| 10/10 [00:11<00:00,  1.19s/it]


In [61]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [62]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.717771,0.143994,21105.0,4.510805,229900
1,ECG1_20k,0.619746,0.125926,675.0,0.407297,20000
2,IOPS1,0.690399,0.000000,206.0,0.124827,8784
3,SMD1,0.517911,0.000000,2694.0,0.309536,28479
4,Occupancy1,0.165914,0.001029,972.0,0.032253,2665
5,ECG1+IOPS1,0.643129,0.108967,881.0,0.592734,28784
6,SMD1+Occupancy1,0.407649,0.000000,3666.0,0.336422,31144
7,ECG1+IOPS1+Occupancy1,0.643716,0.061522,1853.0,0.405919,31449
8,SMD1+ECG1+Occupancy1,0.604010,0.000000,4341.0,0.695640,51144
9,ECG1+IOPS1+SMD1+Occupancy1,0.604448,0.000000,4547.0,1.010408,59928


In [63]:
print(df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']].to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                      Name &      AUC &  Precision@k &  Number of anomalies &     Time &  Number of Windows \\
\midrule
                      ECG1 & 0.717771 &     0.143994 &              21105.0 & 4.510805 &             229900 \\
                  ECG1\_20k & 0.619746 &     0.125926 &                675.0 & 0.407297 &              20000 \\
                     IOPS1 & 0.690399 &     0.000000 &                206.0 & 0.124827 &               8784 \\
                      SMD1 & 0.517911 &     0.000000 &               2694.0 & 0.309536 &              28479 \\
                Occupancy1 & 0.165914 &     0.001029 &                972.0 & 0.032253 &               2665 \\
                ECG1+IOPS1 & 0.643129 &     0.108967 &                881.0 & 0.592734 &              28784 \\
           SMD1+Occupancy1 & 0.407649 &     0.000000 &               3666.0 & 0.336422 &              31144 \\
     ECG1+IOPS1+Occupancy1 & 0.643716 &     0.061522 &               

In [64]:
df.to_csv('Results/Matrix Profile/STUMP-variant3', index=False)

## 5. Online Running: Variant 4: Dynamic Partitioning with Percentiles
The STUMP algorithm is run online, with access to limited subsequences. In this variant, the algorithm only has access to the subsequences of a given (current) batch. The partitioning of the original time series is being done dynamically using percentiles instead of min/max.

In [47]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    x = ts['data']
    
    k = 1
    score = []
    t0 = time()
    for i, batch in enumerate(tqdm(ts['percentile_dynamic_partitioning_batches'], desc='Processing Batch')):

        # If there are not enough points to do at least a window, pad with the last score.
        if len(batch) < window_size:
            score_ = [score[-1]] * len(batch)
        else:
            score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)
            score_ = score_.T[k-1]

            # Because batches are being split in a way that doesn't allow windows to cross batch boundaries, pad predictions to account for lost
            # windows.
            score_ = np.array([score_[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score_) + [score_[-1]]*((ts['slidingWindow']-1)//2))

        score.extend(score_)
    t1 = time()

    # In some combinations of batch size and window size, windows overlap with all closest-distance candidates and cannot be scored.
    # In this case, inf is returned. To fix this, any instances of infinite distances are replaced with zero distance.
    score = [s if s != np.inf else 0 for s in score]
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    # score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1+IOPS1+SMD1+Occupancy1: 100%|██████████| 10/10 [00:11<00:00,  1.10s/it]


In [48]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [49]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.700964,0.168823,21105.0,3.159311,229900
1,ECG1_20k,0.718579,0.382222,675.0,0.272559,20000
2,IOPS1,0.592980,0.000000,206.0,0.093899,8784
3,SMD1,0.498453,0.000000,2694.0,0.348413,28479
4,Occupancy1,0.299461,0.000000,972.0,0.037769,2665
5,ECG1+IOPS1,0.682912,0.303065,881.0,0.390198,28784
6,SMD1+Occupancy1,0.407287,0.000000,3666.0,0.380699,31144
7,ECG1+IOPS1+Occupancy1,0.643717,0.165138,1853.0,0.453855,31449
8,SMD1+ECG1+Occupancy1,0.588531,0.000000,4341.0,0.647888,51144
9,ECG1+IOPS1+SMD1+Occupancy1,0.572299,0.000000,4547.0,0.848507,59928


In [50]:
print(df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']].to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                      Name &      AUC &  Precision@k &  Number of anomalies &     Time &  Number of Windows \\
\midrule
                      ECG1 & 0.700964 &     0.168823 &              21105.0 & 3.159311 &             229900 \\
                  ECG1\_20k & 0.718579 &     0.382222 &                675.0 & 0.272559 &              20000 \\
                     IOPS1 & 0.592980 &     0.000000 &                206.0 & 0.093899 &               8784 \\
                      SMD1 & 0.498453 &     0.000000 &               2694.0 & 0.348413 &              28479 \\
                Occupancy1 & 0.299461 &     0.000000 &                972.0 & 0.037769 &               2665 \\
                ECG1+IOPS1 & 0.682912 &     0.303065 &                881.0 & 0.390198 &              28784 \\
           SMD1+Occupancy1 & 0.407287 &     0.000000 &               3666.0 & 0.380699 &              31144 \\
     ECG1+IOPS1+Occupancy1 & 0.643717 &     0.165138 &               

In [51]:
df.to_csv('Results/Matrix Profile/STUMP-variant4', index=False)